# Машинное обучение, ФКН ВШЭ

# Практическое задание 9. EM-алгоритм

Дата выдачи: 18.02.2021

# Generative model of Labels, Abilities, and Difficulties (GLAD)

В [семинаре 15](https://github.com/esokolov/ml-course-hse/blob/master/2020-spring/seminars/sem15-em.pdf) мы рассмотрели задачу восстановления истинной разметки по меткам от экспертов (которым мы не можем доверять в полной мере, более того, их предсказания могут расходиться).

Рассмотрим следующую вероятностную модель:

$$ p(L, Z | \alpha, \beta) = \prod_{i=1}^{n} \prod_{j=1}^m \sigma(\alpha_j\beta_i)^{[l_{ij}=z_i]}\sigma(-\alpha_j\beta_i)^{1-[l_{ij}=z_i]} p(z_j)$$

где $l_{ij} -$ ответ $j$-го эксперта на задачу $i$, $z_j -$ истинная разметка, $\alpha_i, \beta_j-$ уровень экспертизы и сложность задачи соответственно. Для более подробного описания модели можно прочитать материалы семинара, а также [оригинальную статью](http://papers.nips.cc/paper/3644-whose-vote-should-count-more-optimal-integration-of-labels-from-labelers-of-unknown-expertise.pdf). Априорное распределение положим равномерным: $p(z_i) = 0.5$.

In [1]:
import numpy as np
seed = 0xDEADF00D
np.random.seed(seed)

In [2]:
L = np.load('L.npy')
n, m = L.shape

In [3]:
L.shape

(2000, 20)

**Задание 1. (2 балла)** Реализуйте EM-алгоритм для заданной выше модели. Вы можете воспользоваться предложенными шаблонами или написать свои. 

Обратите внимание, что правдоподобие моделирует не вероятность метки $l_{ij}$ принять значение 1 или 0, а вероятность того, что она равна скрытой переменной $z_i$, т.е. $p(l_{ij} = z_j|z_j, \alpha_j, \beta_i) \neq p(l_{ij} = 1|\alpha_j, \beta_i) $. При этом заранее неизвестно, какая из скрытых переменных соответствует метке 1. Не забывайте, что параметры $\beta_i$ должны быть *положительными*; для этого оптимизируйте $\log \beta$. На M-шаге можете использовать как один шаг градиентного спуска, так и несколько: разумные результаты у вас должны получаться вне зависимости от числа итераций.

Также при работе с вероятностями не забывайте о точности:
1. Используйте логарифмы вероятностей.
2. $\log \sigma(a)$ лучше преобразовать в $\log \sigma(a) = -\log(1 + \exp(-a)) = -\mathrm{softplus}(-a)$
3. Ещё полезные функции: `scipy.special.expit`, `scipy.special.logsumexp`, `np.log1p`

Для отладки может быть полезно проверить градиент с помощью `scipy.optimize.check_grad`.

In [4]:
def softplus(x):
    '''stable version of log(1 + exp(x))'''
    c = (x > 20) * 1.
    return np.log1p(np.exp(x * (1-c)) * (1-c)) + x * c

## Апостериорная вероятность

Апостериорную вероятность $q^*_i(z_i) = p(z_i | l, \alpha, \beta)$ (см. сем 15, страница 7) можно переписать как:
$$p(z_i | l, \alpha, \beta) = \frac{\exp\Big[\log(..1..)\Big]_i}{\sum\limits_{t \in \{0, 1\}}\exp\Big[\log(..2..)\Big]}$$

$$\log(..1..) = \log(p(z_i)) + \sum\limits_j \Big( [l_{ij} = z_i]\log\sigma(\alpha_j \beta_i) +  [l_{ij} \neq z_i]\log\sigma(-\alpha_j \beta_i)\Big)$$

$$\log(..2..) = \log(p(t)) + \sum\limits_j \Big( [l_{ij} = t]\log\sigma(\alpha_j \beta_i) +  [l_{ij} \neq t]\log\sigma(-\alpha_j \beta_i)\Big)$$

$$p(z_i | l, \alpha, \beta) = \frac{\exp\Big[\log(p(z_i)) + \sum\limits_j \Big( [l_{ij} = z_i]\log\sigma(\alpha_j \beta_i) +  [l_{ij} \neq z_i]\log\sigma(-\alpha_j \beta_i)\Big)\Big]}{\sum\limits_{t \in \{0, 1\}}\exp\Big[\log(p(t)) + \sum\limits_j \Big( [l_{ij} = t]\log\sigma(\alpha_j \beta_i) +  [l_{ij} \neq t]\log\sigma(-\alpha_j \beta_i)\Big)\Big]}$$

Значит, нужно применять `softmax`. Раскидаем эту штуку на две суммы:
$$\log p(t) + \sum\limits_j \Big( [l_{ij} = t]\log\sigma(\alpha_j \beta_i) +  [l_{ij} \neq t]\log\sigma(-\alpha_j \beta_i)\Big) =\\
\log p(t) + \sum\limits_j [l_{ij} = t]\log\sigma(\alpha_j \beta_i) + \sum\limits_j [l_{ij} \neq t]\log\sigma(-\alpha_j \beta_i) =$$

Пусть $t = 1$, тогда:
$$= \log p(1) + \sum\limits_j [l_{ij} = 1]\log\sigma(\alpha_j \beta_i) + \sum\limits_j [l_{ij} = 0]\log\sigma(-\alpha_j \beta_i) =\\
\log p(1) + \sum\limits_j l_{ij}\log\sigma(\alpha_j \beta_i) + \sum\limits_j (1 - l_{ij})\log\sigma(-\alpha_j \beta_i)$$

Пусть $t = 0$, тогда:
$$= \log p(0) + \sum\limits_j [l_{ij} = 0]\log\sigma(\alpha_j \beta_i) + \sum\limits_j [l_{ij} = 1]\log\sigma(-\alpha_j \beta_i) =\\
\log p(0) + \sum\limits_j (1 - l_{ij})\log\sigma(\alpha_j \beta_i) + \sum\limits_j l_{ij}\log\sigma(-\alpha_j \beta_i)$$

Итого:
$$\sum\limits_{t \in \{0, 1\}}\exp\Big[\log(p(t)) + \sum\limits_j \Big( [l_{ij} = t]\log\sigma(\alpha_j \beta_i) +  [l_{ij} \neq t]\log\sigma(-\alpha_j \beta_i)\Big)\Big] = \\
\exp\Big[\log p(1) + \sum\limits_j l_{ij}\log\sigma(\alpha_j \beta_i) + \sum\limits_j (1 - l_{ij})\log\sigma(-\alpha_j \beta_i) \Big] + \exp\Big[ \log p(0) + \sum\limits_j (1 - l_{ij})\log\sigma(\alpha_j \beta_i) + \sum\limits_j l_{ij}\log\sigma(-\alpha_j \beta_i)\Big]$$

$$\log\sigma(a) = -\mathrm{softplus}(-a)$$

$$\exp\Big[\log p(1) - \sum\limits_j l_{ij}\mathrm{softplus}(-\alpha_j \beta_i) - \sum\limits_j (1 - l_{ij})\mathrm{softplus}(\alpha_j \beta_i) \Big] + \exp\Big[ \log p(0) - \sum\limits_j (1 - l_{ij})\mathrm{softplus}(-\alpha_j \beta_i) -\sum\limits_j l_{ij}\mathrm{softplus}(\alpha_j \beta_i)\Big]$$

$$\log p(1) - \sum\limits_j l_{ij}\mathrm{softplus}(-\alpha_j \beta_i) - \sum\limits_j (1 - l_{ij})\mathrm{softplus}(\alpha_j \beta_i)$$

$$\sum\limits_j l_{ij}\mathrm{softplus}(-\alpha_j \beta_i) = \langle L_{(i)}, \mathrm{softplus}(-\beta \alpha^T)_{(i)} \rangle = \langle L_{(i)}, \mathrm{softplus}(-\beta \alpha^T)^{T_{(i)}} \rangle$$
Это поэлементное умножение `np.multiply` + аггрегация по сумме по столбцам (`axis=1`). Тогда:

$$\exp\Big[\log p(1) - \langle L_{(i)}, \mathrm{softplus}(-\beta \alpha^T)_{(i)} \rangle - \langle 1 - L_{(i)}, \mathrm{softplus}(\beta \alpha^T)_{(i)} \rangle  \Big] + \exp\Big[ \log p(0) - \langle 1 - L_{(i)}, \mathrm{softplus}(-\beta \alpha^T)_{(i)} \rangle - \langle L_{(i)}, \mathrm{softplus}(\beta \alpha^T)_{(i)} \rangle \Big]$$

Т.е. программное решение должно строиться на основе адамарого произведения матрицы $L$ на матрицу $\mathrm{softplus}(\pm\beta \alpha^T)$.

In [5]:
from numpy import multiply as hadamar
from scipy.special import softmax

def posterior(alpha, beta, L):
    """ Posterior over true labels z p(z|l, \alpha, \beta)
    Args:
        alpha: ndarray of shape (n_experts).
        beta: ndarray of shape (n_problems).
        L: ndarray of shape (n_problems, n_experts).
    """
    
    log_half = np.log(0.5)
    
    bsz = (beta.shape[0], 1)
    asz = (1, alpha.shape[0])
    ba = beta.reshape(bsz) @ alpha.reshape(asz)
    
    softp_mat_plus = softplus(ba)
    softp_mat_minus = softplus(-ba)
    
    class_0 = log_half - hadamar(1 - L, softp_mat_minus).sum(axis=1) - hadamar(L, softp_mat_plus).sum(axis=1)
    class_1 = log_half - hadamar(L, softp_mat_minus).sum(axis=1) - hadamar(1 - L, softp_mat_plus).sum(axis=1)
    
    stacked = np.vstack((class_0, class_1))
    return softmax(stacked, axis=0)

## Логарифм правдоподобия

$$p(l_{ij} = z_i | \alpha_j, \beta_i) = \sigma(\alpha_j \beta_i)$$
$$\log p(l_{ij} = z_i | \alpha_j, \beta_i) = \log\sigma(\alpha_j \beta_i)$$

Пользуясь инверсией вероятности и тем, что $1 - \sigma(x) = \sigma(-x)$,

$$p(l_{ij} \neq z_i | \alpha_j, \beta_i) = 1 - \sigma(\alpha_j \beta_i) = \sigma(-\alpha_j \beta_i)$$
$$\log p(l_{ij} \neq z_i | \alpha_j, \beta_i) = \log(\sigma(-\alpha_j \beta_i))$$

Рассмотрим матрицу $\hat{L}$, такую что $\hat{L}_{i,j} = [l_{ij} = z_i]$. Тогда:

$$\hat{L}_{i,j} \text{ перейдет в } \hat{A}\ :\ \hat{A}_{i,j} = \hat{L}_{i,j}\log\sigma(\alpha_j \beta_i) + (1 - \hat{L}_{i,j})\log(\sigma(-\alpha_j \beta_i)),$$

Что просто является суммой двух матриц, полученных с помощью адамарого произведения. Таким образом,

$$\hat{A} = -\hat{L}\circ\mathrm{softplus}(-\beta \alpha^T) - (1 - \hat{L})\circ\mathrm{softplus}(\beta \alpha^T)$$

In [6]:
from scipy.special import expit
def log_likelihood(alpha, beta, L, z):
    """ p(l=z|z, \alpha, \beta)
    Args:
        alpha: ndarray of shape (n_experts).
        beta: ndarray of shape (n_problems).
        L: ndarray of shape (n_problems, n_experts).
        z: ndarray of shape (n_problems).
    """
    
    bsz = (beta.shape[0], 1)
    asz = (1, alpha.shape[0])
    ba = beta.reshape(bsz) @ alpha.reshape(asz)
    
    L_hat = (L == z.reshape(bsz)).astype(int)  # np.equal
    A_hat = -hadamar(L_hat, softplus(-ba)) - hadamar(1 - L_hat, softplus(ba))
    
    return A_hat.sum()

In [7]:
# O = np.array([ [1, 0, 1],
#                [1, 1, 1],
#                [1, 1, 0],
#                [0, 0, 1],
#                [0, 1, 0]
#              ])
# a = np.array([-2, 10, 4])
# b = np.array([2, 7, 10, 4, 8])
# z = np.array([0, 1, 1, 0, 0])

# log_likelihood(a, b, O, z)
# np.sum(posterior(a, b, O), axis=0)

## Градиенты
$$\frac{\partial}{\partial \alpha_j} \mathbb{E}_{q^*} \log p(z, l | \alpha, \beta) =\\
\sum\limits_i \sum\limits_{t \in \{0, 1\}} q^*_i(t) \beta_i \Big( [l_{ij} = t]\sigma(-\alpha_j \beta_i) - [l_{ij} \neq t] \sigma(\alpha_j \beta_i) \Big) =\\
\sum\limits_i \sum\limits_{t \in \{0, 1\}} q^*_i(t) \beta_i \Big( [l_{ij} = t]-[l_{ij} = t]\sigma(\alpha_j \beta_i) - [l_{ij} \neq t] \sigma(\alpha_j \beta_i) \Big) = \\
\sum\limits_i \sum\limits_{t \in \{0, 1\}} q^*_i(t) \beta_i \Big( [l_{ij} = t]-\sigma(\alpha_j \beta_i)\Big[[l_{ij} = t] +  [l_{ij} \neq t]\Big] \Big) =\\
\sum\limits_i \sum\limits_{t \in \{0, 1\}} q^*_i(t) \beta_i ([l_{ij} = t] - \sigma(\alpha_j \beta_i))$$

$$\frac{\partial}{\partial \alpha_j} \mathbb{E}_{q^*} \log p(z, l | \alpha, \beta) =
\sum\limits_i \sum\limits_{t \in \{0, 1\}} q^*_i(t) \beta_i ([l_{ij} = t] - \sigma(\alpha_j \beta_i))$$

$$\frac{\partial}{\partial \beta_i} \mathbb{E}_{q^*} \log p(z, l | \alpha, \beta) =
\sum\limits_j \sum\limits_{t \in \{0, 1\}} q^*_i(t) \alpha_j ([l_{ij} = t] - \sigma(\alpha_j \beta_i))$$

Еще немного распишем и воспользуемся структурой таблицы распределения $q*$, реализованной в функции `posterior`, а именно: строка `q[0]` соответствует вероятностям нулевого класса для задач (по столбцам); строка `q[1]` соответствует вероятностям класса 1. Сумма по столбцам (`axis=0`) равна 1 для каждой задачи.

$$\frac{\partial}{\partial \alpha_j} \mathbb{E}_{q^*} \log p(z, l | \alpha, \beta) = \sum\limits_i \sum\limits_{t \in \{0, 1\}} q^*_i(t) \beta_i ([l_{ij} = t] - \sigma(\alpha_j \beta_i)) =\\
\sum\limits_i \Big( q^*_i(0) \beta_i ([l_{ij} = 0] - \sigma(\alpha_j \beta_i)) + q^*_i(1) \beta_i ([l_{ij} = 1] - \sigma(\alpha_j \beta_i))\Big) =\\
\sum\limits_i \beta_i\Big( q^*_i(0) [l_{ij} = 0] - q^*_i(0)\sigma(\alpha_j \beta_i) + q^*_i(1)[l_{ij} = 1] - q^*_i(1)\sigma(\alpha_j \beta_i)\Big) =\\
\sum\limits_i \beta_i\Big( q^*_i(0) [l_{ij} = 0] - \underbrace{\big[q^*_i(0) + q^*_i(1)\big]}_{=1\ \forall\ i} \sigma(\alpha_j \beta_i) + q^*_i(1)[l_{ij} = 1]\Big) =\\
\sum\limits_i \beta_i\Big( q^*_i(0) [l_{ij} = 0] - \sigma(\alpha_j \beta_i) + q^*_i(1)[l_{ij} = 1]\Big)$$

Обозначим $\Theta_{i,j} = \Big(q^*_i(0) [l_{ij} = 0] + q^*_i(1)[l_{ij} = 1]\Big)$, $\Theta \in \mathbb{R}^{np \times ne}$,  тогда:
$$\frac{\partial}{\partial \alpha_j} \mathbb{E}_{q^*} \log p(z, l | \alpha, \beta) = \sum\limits_i \beta_i\Theta_{i,j} - \sum\limits_i\beta_i\sigma(\alpha_j \beta_i)$$

Аналогичным образом,
$$\frac{\partial}{\partial \beta_i} \mathbb{E}_{q^*} \log p(z, l | \alpha, \beta) = \sum\limits_j \alpha_j\Theta_{i,j} - \sum\limits_j\alpha_j\sigma(\alpha_j \beta_i)$$

В конечном итоге,
$$\nabla_\alpha \mathbb{E}_{q^*} \log p(z, l | \alpha, \beta) = \Big[\sum\limits_i^{np}\beta_i \Theta_{i,0} -\sum\limits_i^{np}\beta_i \sigma(\alpha_0 \beta_i),\ \ldots,\ \sum\limits_i^{np}\beta_i \Theta_{i,ne} -\sum\limits_i^{np}\beta_i \sigma(\alpha_{ne} \beta_i)\Big]^T \in \mathbb{R}^{ne}$$
$$\nabla_\beta \mathbb{E}_{q^*} \log p(z, l | \alpha, \beta) = \Big[\sum\limits_j^{ne} \alpha_j\Theta_{0,j} -\sum\limits_j^{ne}\alpha_j \sigma(\alpha_j \beta_0),\ \ldots,\ \sum\limits_j^{ne} \alpha_j\Theta_{ne,j} -\sum\limits_j^{ne}\alpha_j \sigma(\alpha_j \beta_{np})\Big]^T \in \mathbb{R}^{np}$$

$$\nabla_\alpha \mathbb{E}_{q^*} \log p(z, l | \alpha, \beta) = \sum\limits_i^{np}\beta_i\Big[\Theta_{i,0},\ \ldots,\ \Theta_{i,ne}\Big]^T -\sum\limits_i^{np}\beta_i \Big[\sigma(\alpha_0 \beta_i),\ \ldots,\ \sigma(\alpha_{ne} \beta_i)\Big]^T \text{ (столбцы по $i$-ой строке)}$$
$$\nabla_\beta \mathbb{E}_{q^*} \log p(z, l | \alpha, \beta) =\sum\limits_j^{ne} \alpha_j\Big[\Theta_{0,j},\ \ldots,\ \Theta_{np,j}]^T-\sum\limits_j^{ne}\alpha_j\Big[ \sigma(\alpha_j \beta_0),\ \ldots,\ \sigma(\alpha_j \beta_{np})\Big]^T \text{ (строки по $j$-му столбцу)}$$

Очевидно, градиенты представляют собой линейные комбинации векторов, и записать их можно так:

$$\nabla_\alpha \mathbb{E}_{q^*} \log p(z, l | \alpha, \beta) = \Big(\Theta-\sigma(\beta\alpha^T)\Big)^T\beta$$
$$\nabla_\beta \mathbb{E}_{q^*} \log p(z, l | \alpha, \beta) = \Big(\Theta-\sigma(\beta\alpha^T)\Big)\alpha$$

Осталось лишь понять, как устроена $\Theta$. Для начала,
$$\Theta_{i,j} = \Big(q^*_i(0) Ind_{i,j}^0 + q^*_i(1)(1 - Ind_{i,j}^0)\Big),\ Ind_{i,j}^0 = [L_{i,j} = 0]$$
Рассмотрим $q^*_i(0) Ind_{i,j}^0$ и заметим, что устройство соответствующей матрицы такое: ее $j$-ый столбец $-$ это $Ind^0_j \circ q[0, :]$, где $Ind^0_j$ $-$ $j$-ый столбец матрицы $Ind^0$, а $q[0, :]$ $-$ нулевая строка таблицы распределений $q*$. Аналогично и со вторым слагаемым. Такое можно реализовать через `np.multiply` $-$ см. пример ниже. Далее в коде используется алиас `hadamar`.

In [8]:
O = np.array([[2, 3, 4],
             [1, 1.5, 2],
             [0.5, 0.75, 1]])
arr = np.array([2, 4, 8]).reshape(3, 1)
np.multiply(O, arr)

array([[4., 6., 8.],
       [4., 6., 8.],
       [4., 6., 8.]])

In [9]:
def alpha_grad_lb(alpha, beta, L, q):
    """ Gradient of lower bound wrt alpha
    Args:
        alpha: ndarray of shape (n_experts).
        beta: ndarray of shape (n_problems).
        L: ndarray of shape (n_problems, n_experts).
        q: ndarray of shape (2, n_problems).
            q[0] - label 0
            q[1] - label 1
    """
    
    bsz = (beta.shape[0], 1)
    asz = (1, alpha.shape[0])
    ba = beta.reshape(bsz) @ alpha.reshape(asz)
    Ind_0 = (L == 0).astype(int)
    theta = hadamar(Ind_0, q[0].reshape(bsz)) + hadamar(1 - Ind_0, q[1].reshape(bsz))
    return (theta-expit(ba)).T @ beta


def logbeta_grad_lb(alpha, beta, L, q):
    """ Gradient of lower bound wrt alpha
    Args:
        alpha: ndarray of shape (n_experts).
        beta: ndarray of shape (n_problems).
        L: ndarray of shape (n_problems, n_experts).
        q: ndarray of shape (2, n_problems).
            q[0] - label 0
            q[1] - label 1
    """
    
    bsz = (beta.shape[0], 1)
    asz = (1, alpha.shape[0])
    ba = beta.reshape(bsz) @ alpha.reshape(asz)
    Ind_0 = (L == 0).astype(int)
    theta = hadamar(Ind_0, q[0].reshape(bsz)) + hadamar(1 - Ind_0, q[1].reshape(bsz))
    return (theta-expit(ba)) @ alpha


def lower_bound(alpha, beta, L, q):
    """ Lower bound
    Args:
        alpha: ndarray of shape (n_experts).
        beta: ndarray of shape (n_problems).
        L: ndarray of shape (n_problems, n_experts).
        q: ndarray of shape (2, n_problems).
    """
    pass

In [10]:
def em(L, n_steps=1000, lr=1e-3):
    # initialize parameters
    alpha, logbeta = np.random.randn(m), np.random.randn(n)
    q = np.ones((2, len(logbeta))) * 0.5

    for step in range(n_steps):
        beta = np.exp(logbeta)
        # E-step
        q = posterior(alpha, beta, L)
        # M-step
        logbeta += lr * logbeta_grad_lb(alpha, logbeta, L, q)
        alpha += lr * alpha_grad_lb(alpha, logbeta, L, q)

    return alpha, np.exp(logbeta), q

In [11]:
alpha, beta, q = em(L)

**Задание 2. (1 балл)** Загрузите настоящую разметку. Посчитайте `accuracy` разметки, полученной с помощью обычного голосования по большинству среди экспертов, и сравните его с качеством, полученным с помощью EM-алгоритма. Помните, что алгоритму не важно, какая метка 0, а какая 1, поэтому если получите качество <0.5, то просто поменяйте метки классов (не забудьте также поменять знак у $\alpha$). 

In [12]:
from sklearn.metrics import accuracy_score
y = np.load('y.npy')
majority_class = (L.mean(axis=1) >= 0.5).astype(int)
accuracy_score(y, majority_class)

0.892

In [13]:
max(accuracy_score(y, (q[1] < 0.5)), accuracy_score(y, (q[0] < 0.5)))

0.955

*Теперь мне грустно, что я потратил 8.5 часов на алгоритм, который улучшил качество всего на 0.06 ((((*
# (∩ ￣ー￣)⊃ ✳✨✳✨✳✨✳

**Задание 3. (0.5 балла)** Попробуйте проинтерпретировать полученные коэфициенты $\alpha$. Есть ли в выборке эксперты, которые намеренно голосуют неверно? Как это можно понять по альфам? Продемонстрируйте, что эксперты действительно чаще голосуют за неверный класс. Постройте график зависимости доли верно размеченных экспертом объектов от коэффициента $\alpha$. Прокомментируйте результаты.

In [14]:
alpha

array([-1.00564337, -1.13198555, -7.06877417, -0.86472804, -0.98360413,
       -6.94800087, -0.97709992, -6.41718119,  6.72142754, -1.03751965,
        7.07653027,  6.92312786, -6.72793314, -0.89754776, -1.02643943,
       -0.86183692, -1.0346459 , -6.59548809,  6.45215403, -6.51211908])

In [15]:
import matplotlib.pyplot as plt

L_hat = (L == y.reshape((2000, 1))).astype(int) 

plt.scatter(alpha, L_hat.mean(axis=0), color='royalblue')
plt.xlabel('alpha')
plt.ylabel('accuracy')
plt.title('Accuracy - Alpha')
plt.show()

<Figure size 640x480 with 1 Axes>

Такие эксперты есть, и у них отрицательное `alpha`. Данных не очень много, поэтому на графике можно говорить лишь о точках по краям. У экспертов с `aplha ~ 6` высокий `accuracy`, у тех, у кого `alpha ~ -6` -- наборот. Если присмотреться, можно заметить, что если у самых топовых экспертов средняя `accuracy` свыше 0.9, то у самых плохих -- ниже 0.1. Кажется, что если их сложить, получим примерно 1.0.

In [16]:
top = np.argwhere(alpha > 3)
bottom = np.argwhere(alpha < 0)
L_top = L_hat[:, top]
L_bottom = L_hat[:, bottom]
np.mean(L_top.mean(axis=0)) + np.mean(L_bottom.mean(axis=0))

0.8598125000000001

In [17]:
np.mean(L_top.mean(axis=0)), np.mean(L_bottom.mean(axis=0))

(0.08574999999999999, 0.7740625000000001)

То есть, такие эксперты явно есть, и они отмечаются моделью как вредные, ведь даже эксперты с небольшим `alpha` около нуля выполняюс разметку не очень плохо (если бы человек случайно решал тест, где каждый вопрос имел бы два варианта ответа, то его `accuracy` было бы 0.5, что равносильно подбрасыванию монеты), что делает их просто не очень хорошими экспертами. Поэтому крайне низкое `accuracy` выглядит как-то контринтуитивно. В общем, такие эксперты действительно голосуют в основном за неверный класс.

**Задание 4. (бонус, 2 балла)**  Как уже было замечено выше, модели не важно, какой класс 1, а какой 0. Скажем, если все эксперты оказались максимально противными и ставят метку с точностью наоборот, то у вас будет полная согласованность между экспертами, при этом невозможно понять правильно они разметили выборку или нет, смотря только на такую разметку. Чтобы избежать этого, можно включать в выборку вопрос с заведомо известным ответом, тогда вы сможете определить, ставит ли эксперт специально неверные метки.

Чтобы обощить данную модель на случай заданий с заведомо известной меткой, достаточно не делать для них E-шаг, а всегда полагать апостериорное распределение вырожденным в истинном классе. Реализуйте данную модель и используйте истинную разметку *для нескольких* задач из обучения. Проинтерпретируйте полученные результаты.

In [18]:
# def triv_posterior(y):
#     class_1 = y
#     class_0 = np.ones((y.shape)) - y
#     return np.vstack((class_0, class_1))

# triv_posterior(np.array([0, 1, 1, 1, 0, 0, 1]))

# def triv_em(L, q, n_steps=1000, lr=1e-3):
#     # initialize parameters
#     n, m = L.shape
#     alpha, logbeta = np.random.randn(m), np.random.randn(n)

#     for step in range(n_steps):
#         beta = np.exp(logbeta)
#         # M-step
#         logbeta += lr * logbeta_grad_lb(alpha, beta, L, q)
#         alpha += lr * alpha_grad_lb(alpha, beta, L, q)

#     return alpha, np.exp(logbeta)

In [19]:
# q = triv_posterior(y)
# inds = np.random.choice(2000, size=int(0.7 * 2000), replace=False)
# alpha, beta = triv_em(L[inds], q[:, inds])

# Выравнивание слов (Word Alignment)

EM-алгоритм также применяют на практике для настройки параметров модели выравнивания слов, более сложные модификации которой используются в статистическом машинном переводе. Мы не будем подробно обсуждать применение word alignment для перевода и ограничимся следующей целью: пусть у нас есть параллельный корпус из предложений на исходном языке и их переводов на целевой язык (в этом задании используются английский и чешский соответственно). 

Первая задача — определить с помощью этого корпуса, как переводится каждое отдельное слово на целевом языке. Вторая задача — для произвольной пары из предложения и его перевода установить, переводом какого слова в исходном предложении является каждое слово в целевом предложении. Оказывается, у обеих задач существует элегантное и эффективное решение при введении правильной вероятностной модели: в этой части задания вам предстоит его реализовать и оценить результаты работы. Но обо всём по порядку :)

---

Перед тем, как заниматься машинным обучением, давайте разберёмся с данными и метриками в интересующей нас задаче. В ячейке ниже загружается и разархивируется параллельный английско-чешский корпус, в котором есть разметка выравнивания слов. Нетрудно заметить, что формат XML-файла, использованный его авторами, не вполне стандартный: нет готовой команды , которая позволила бы получить список пар предложений вместе с выравниваниями. Это значит, что нужно разобраться с форматом и написать парсер самостоятельно, используя встроенные средства Python, например, модуль [xml](https://docs.python.org/3.7/library/xml.html).

In [20]:
# %%bash
# wget -q https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-1804/CzEnAli_1.0.tar.gz -O CzEnAli_1.0.tar.gz
# mkdir -p data
# tar -xzf CzEnAli_1.0.tar.gz -C data/
# head -n 20 data/merged_data/project_syndicate/project_syndicate_bacchetta1.wa

**Задание -2. (0.5 балла)** Реализуйте функцию `extract_sentences`, которая принимает на вход путь к файлу с XML-разметкой, используемой в этом датасете, и возвращает список параллельных предложений, а также список из «уверенных» (sure) и «возможных» (possible) пар выравниваний. Отправьте вашу реализацию в Яндекс.Контест, чтобы убедиться в её корректности; в следующей ячейке ноутбука соберите все пары размеченных предложений из датасета в два списка `all_sentences` (список `SentencePair`) и `all_targets` (список LabeledAlignment).

Здесь и далее соблюдайте сигнатуры функций и пользуйтесь объявленными в модуле `preprocessing.py` классами для организации данных. Стоит заметить, что предложения уже токенизированы (даже отделена пунктуация), поэтому предобработку текстов совершать не нужно. Обратите внимание на формат хранения выравниваний: нумерация начинается с 1 (в таком виде и нужно сохранять), первым в паре идёт слово из англоязычного предложения.

In [21]:
import glob
from codes.preprocessing import extract_sentences

sentences = []
targets = []

for file in glob.glob('data\*\*\*.wa'):
    tmp_s, tmp_t = extract_sentences(file)
    sentences.extend(tmp_s)
    targets.extend(tmp_t)

**Задание -1. (0.5 балла)** Реализуйте функции `get_token_to_index` и `tokenize_sents` из модуля `preprocessing.py`, постройте словари token->index для обоих языков и постройте список из `TokenizedSentencePair` по выборке. Реализации функций также отправьте в Яндекс.Контест.

In [22]:
from codes.preprocessing import get_token_to_index, tokenize_sents

t_idx_src, t_idx_tgt = get_token_to_index(sentences)
tokenized_sentences = tokenize_sents(sentences, t_idx_src, t_idx_tgt)

В качестве бейзлайна для этой задачи мы возьмём способ выравнивания слов по коэффициенту Дайса: слово в исходном языке является переводом слова на целевом языке, если они часто встречаются в одних и тех же предложениях и редко встречаются по отдельности. 

Математически это записывается по аналогии с мерой Жаккара: пусть $c(x,y)$ — число параллельных предложений, в которых есть и $x$ (на исходном языке), и $y$ (на целевом языке), а $c(x)$ и $c(y)$ — суммарное количество предложений, в которых встречается слово $x$ и $y$ соответственно. Тогда $\textrm{Dice}(x,y)=\frac{2 \cdot c(x,y)}{c(x) + c(y)}$ — характеристика «похожести» слов $x$ и $y$. Она равна 1, если слова встречаются только в контексте друг друга (не бывает предложений только со словом $x$ без $y$ в переводе и наоборот), равна 0, если слова никогда не встречаются в параллельных предложениях и находится между пороговыми значениями в остальных случаях.

В файле `models.py` описан абстрактный класс `BaseAligner`, наследником которого должны являться все модели в задании, а также приведён пример реализации `DiceAligner` выравнивания слов описанным выше путём. Ниже вы можете увидеть, как применять эту модель.

In [23]:
from codes.models import DiceAligner

baseline = DiceAligner(len(t_idx_src), len(t_idx_tgt), threshold=0.01)
baseline.fit(tokenized_sentences)

Чтобы оценить качество модели выравнивания, пользуясь имеющейся разметкой, существует ряд автоматических метрик. Они подразумевают, что в разметке есть два вида выравниваний — «уверенные» (sure) и «возможные» (possible). Обозначим для конкретного предложения первое множество выравниваний $S$, второе — $P$, а предсказанные выравнивания — $A$; причём, в отличие от разметки в файле, $S\subseteq P$. Тогда можно предложить три метрики, используя только операции над этими множествами:

Precision $=\frac{|A\cap P|}{|A|}$. Отражает, какая доля предсказанных нами выравниваний вообще корректна; если мы дадим в качестве ответа все возможные пары слов в предложении, эта метрика сильно просядет.

Recall $=\frac{|A\cap S|}{|S|}$. Эта метрика показывает, какую долю «уверенных» выравниваний мы обнаружили. Если мы попытаемся сделать слишком консервативную модель, которая выдаёт 0 или 1 предсказание на нетривиальных предложениях, полнота получится крайне низкая. 

Alignment Error Rate (AER) $=1-\frac{|A\cap P|+|A\cap S|}{|A|+|S|}$. Метрика является комбинацией двух предыдущих и отслеживает общее качество работы системы, штрафуя оба описанных выше вида нежелаемого поведения модели. 

**Задание 0. (0.5 балла)** Реализуйте функции compute_precision, compute_recall, compute_aer из модуля metrics.py. Оцените качество бейзлайнового метода. Обратите внимание, что нужно использовать микро-усреднение во всех функциях: необходимо просуммировать числитель и знаменатель по всем предложениям и только потом делить.

In [24]:
from codes.metrics import compute_aer

compute_aer(targets, baseline.align(tokenized_sentences))

0.8115275584918071

Теперь мы можем перейти к базовой вероятностной модели для выравнивания слов. Пусть $S=(s_1,\ldots,s_n)$ исходное предложение, $T=(t_1,\ldots,t_m)$ — его перевод. В роли латентных переменных будут выступать выравнивания $A=(a_1,\ldots,a_m)$ каждого слова в целевом предложении, причём $a_i\in\{1,\ldots,n\}$ (считаем, что каждое слово в $t$ является переводом какого-то слова из $s$). *Параметрами* модели является матрица условных вероятностей перевода: каждый её элемент $\theta(y|x)=p(y|x)$ отражает вероятность того, что переводом слова $x$ с исходного языка на целевой является слово $y$ (нормировка, соответственно, совершается по словарю целевого языка). Правдоподобие латентных переменных и предложения на целевом языке в этой модели записывается так:

$$
p(A,T|S)=\prod_{i=1}^m p(a_i)p(t_i|a_i,S)=\prod_{i=1}^m \frac{1}{n}\theta(t_i|s_{a_i}).
$$ 

**Задание 1. (2 балла)** Выведите шаги EM-алгоритма для этой модели, а также получите выражение для подсчёта нижней оценки правдоподобия ($\mathcal{L}$ в обозначениях лекции и семинара). **Обратите внимание, что на M-шаге нужно найти аналитический максимум по параметрам.**

Для начала перепишу формулу.

$$
p(t,z|s)=\prod_{i=1}^m p(z_i)p(t_i|z_i,s)=\prod_{i=1}^m \frac{1}{n}\theta(t_i|s_{z_i})
$$ 

$$
p(z_i) = \frac{1}{n}, \text{ так как слово $t_i$ может равновероятно выступать переводом любого слова из $s$, тогда } p(t_i|z_i,s)=\theta(t_i|s_{z_i}) (= p(t_i|s_{z_i}))
$$

Слово $s_{z_i}$ с исходного языка переводится на $t_i$.

$$
\Theta\ :\ \Theta_{i,j} = p(t_i|s_j),\ \text{так что если}\ j=z_i, \text{то } \Theta_{i,z_i} = p(t_i|s_{z_i}),\ \Theta \in \mathbb{R}^{m\times n}
$$

### E-шаг
$q^*(z) = p(z|x,\theta^{old}) = p(z|t,s)$. Здесь так себе нотация или черт знает, как это назвать, но под правой частью подразумеваем $p(z|t,s, \Theta^{old})$.

$$
q^*(z_i) = p(z_i|x_i,\theta^{old}) = p(z_i|t_i,s) = \text{ [формула Байеса для $p(z_i|t_i)$ ] } = \frac{p(z_i|s) \cdot p(t_i|z_i,s)}{p(t_i|s)}
$$

Здесть $p(z_i|t_i) = p(z_i)$, так как распределение $z$ от предложения $s$ не зависит -- (?). Тогда

$$
q^*(z_i) = \frac{\theta(t_i|s_{z_i})}{np(t_i|s)}
$$

$p(t_i|s)$ обозначает вероятность того, что $t_i$ является переводом какого-нибудь слова из $s$. Тогда можно расписать: $p(t_i|s) = \sum\limits_{j=1}^n p(t_i|s_j) = \sum\limits_{j=1}^n\theta(t_i|s_j)$.

$$
q^*(z_i) = \frac{\theta(t_i|s_{z_i})}{n\sum\limits_{j=1}^n\theta(t_i|s_j)} \longleftarrow \frac{\Theta^{old}_{i,z_i}}{n\sum\limits_{j=1}^n\Theta^{old}_{i,j}}
$$

### M-шаг

$$
\mathbb{E}_{z\sim q^*(z)} \log p(t,z|s) =
\sum\limits_{i=1}^m q^*(z_i)\log(\frac{1}{n}\theta(t_i|s_{z_i})) =\\
\sum\limits_{i=1}^m q^*(z_i)\log\theta(t_i|s_{z_i}) - \sum\limits_{i=1}^m q^*(z_i)\log n =\\
\sum\limits_{i=1}^m q^*(z_i)\log\theta(t_i|s_{z_i}) - \log n =\\
\sum\limits_{i=1}^m  \frac{\Theta^{old}_{i,z_i}}{n\sum\limits_{j=1}^n\Theta^{old}_{i,j}} \log \Theta^{old}_{i,z_i} - \log n
$$

Далее преобразование в матричный вид и дифференцирование по $\Theta$. Я не совсем понимаю, как можно такое сделать, учитывая $j$ и $z_i$, поэтому в этой ситуации мы просто, наша, это самое, мы уже здесь, наши полномочия всё, окончены.

**Задание 2. (2.5 балла)** Реализуйте все методы класса `WordAligner` в соответствии с полученными вами формулами. Протестируйте вашу реализацию через Яндекс.Контест, а здесь обучите модель и посчитайте её AER на истинной разметке. Чтобы предсказать выравнивание для пары предложений в этой модели, следует выбирать в соответствие для слова в целевом предложении с индексом $i$ позицию, соответствующую максимуму апостериорного распределения $p(a_i|T,S)$.

In [ ]:
from codes.models import WordAligner

word_aligner = WordAligner(len(t_idx_src), len(t_idx_tgt), 20)
word_aligner.fit(tokenized_sentences);

# ༼つ ಠ益ಠ༽つ ─=≡ΣO))

Заметим, что таблицу вероятностей перевода можно использовать и саму по себе для построения словарей. Пример работы показан ниже: метод хоть и работает, но мягко говоря, неидально — слишком мало данных.

In [ ]:
idx_token_tgt = {index:token for token, index in t_idx_tgt.items()}

In [ ]:
[idx_token_tgt[i] for i in word_aligner.translation_probs[t_idx_src['Mr']].argsort()[-3:]]

In [ ]:
[idx_token_tgt[i] for i in word_aligner.translation_probs[t_idx_src['Mrs']].argsort()[-3:]]

In [ ]:
[idx_token_tgt[i] for i in word_aligner.translation_probs[t_idx_src['water']].argsort()[-3:]]

In [ ]:
[idx_token_tgt[i] for i in word_aligner.translation_probs[t_idx_src['depended']].argsort()[-3:]]

In [ ]:
[idx_token_tgt[i] for i in word_aligner.translation_probs[t_idx_src['on']].argsort()[-3:]]

**Задание 3. (0.5 балла)** Мы смогли получить матрицу условных вероятностей перевода исходного языка в целевой. Можно ли, пользуясь этой матрицей и ещё какими-то статистиками по параллельному корпусу, получить вероятности перевода целевого языка в исходный? Реализуйте такой метод и приведите ниже пример его работы, показав пару удачных переводов.

In [ ]:
# (>ω<)ノ—==ΞΞ☆*✲ﾟ

**Задание 4. (0.5 балла)** Визуализируйте полученные выравнивания для нескольких предложений в виде heatmap: по одной из осей располагаются токены исходного текста, по другой — токены его перевода, на пересечении позиций $i$ и $j$ — 0 либо 1 в зависимости от того, является ли в обученной модели $a_i$ равным $j$. Можете ли вы их проинтерпретировать? Постройте аналогичный график, но без дискретизации, а визуализируя напрямую апостериорное распределение. Можете ли вы найти ситуации, в которых модель не уверена, переводом какого слова является слово $i$?

In [ ]:
# (•̀ 3 •́)━★☆.*･｡ﾟ

Заметим, что при задании модели мы сделали довольно сильное предположение о том, что вероятности выбора слова для выравнивания никак не зависят от позиции слова в целевом предложении. Можно сделать эти вероятности настраиваемыми параметрами, получив прямоугольную матрицу $\phi_{m,n}(j|i)=p(a_i=j|m,n)$ для каждой пары длин предложений $m,n$: по-прежнему мы получаем распределение над индексами в исходном предложении. Тогда модель приобретает вид
$$
p(A,T|S)=\prod_{i=1}^m p(a_i|m,n)p(t_i| a_i, S)=\prod_{i=1}^m \phi_{m,n}(a_i|i)\theta(t_i|s_{a_i}).
$$

**Задание 5. (1.5 балла)** Выведите шаги EM-алгоритма для этой модели, а также получите выражение для подсчёта нижней оценки правдоподобия.

ଘ(๑˃̵ᴗ˂̵)━☆ﾟ.*･｡ﾟ

**Задание 6. (2 балла)** Реализуйте все методы класса `WordPositionAligner`, протестируйте их корректность через Яндекс.Контест. Обучите модель, оцените её качество на истинной разметке и сравните его с качеством предыдущей более простой модели. Проиллюстрируйте влияние стартовых параметров на результат, проинициализировав эту модель параметрами модели из задания 2 (важно, чтобы суммарное число эпох обучения в обоих сценариях оставалось тем же).

In [ ]:
from codes.models import WordPositionAligner
# (≧ ◡ ≦)━★☆.*･｡ﾟ

**Задание 7. (1 балл)** В предыдущих пунктах мы никак не заостряли внимание на предобработке текстов, что может негативно влиять на результаты обученной модели. Например, сейчас метод выравнивания учитывает регистр, а слова на чешском языке вдобавок обладают богатой морфологией и большим количеством диакритических знаков. Если сократить количество параметров модели (различных слов), можно ускорить обучение и добиться лучших результатов, потому что статистики по словам будут считаться по большему числу параллельных предложений.

Примените к исходным данным [Unicode-нормализацию](https://en.wikipedia.org/wiki/Unicode_equivalence#Normalization), приведите их к нижнему регистру и обучите модель выравнивания заново. Сравните качество и скорость обучения с предыдущими результатами и сделайте выводы. Если вы найдете в данных ещё какие-то проблемы, которые можно исправить более грамотной предобработкой, также продемонстрируйте, как их решение влияет на качество.

**Важно:** здесь и далее в процессе обработки данных у вас может получаться, что из тестовых данных будут удалены предложения из-за отсутствия слов в словаре. Если такое всё же произошло, для корректности сравнения считайте AER вашей модели на удалённых предложениях равным 1.

In [ ]:
# (੭•̀ω•́)੭̸*✩⁺˚

**Задание 7. (бонус, до 3 баллов)** 

Улучшите качество получившейся системы настолько, насколько сможете. За каждые 5 процентов, на которые AER на тех же данных получается меньше, чем минимум ошибки всех предыдущих моделей, вы получите по 1 бонусному баллу.

Ниже приведены несколько идей, которые могут помочь вам повысить 

* Модифицировать модель: как вы можете понять, недостатком второго реализованного вами подхода является избыточное число параметров из-за необходимости подерживать отдельную матрицу для каждой различной пары длин предложений в корпусе. В статье https://www.aclweb.org/anthology/N13-1073.pdf приведён способ снижения числа параметров, задающих априорное распределение позиций выравнивания, который позволяет в десять раз быстрее обучать модель и получать лучшее качество.
* Агрегация по двум направлениям: в статье https://www.aclweb.org/anthology/J03-1002/ утверждается, что асимметричность выравниваний вредит качеству, потому что из-за выбранной модели одному слову в целевом предложении не может соответствовать два слова в исходном предложении. Для решения этой проблемы (и улучшения метрик, разумеется) авторы предлагают несколько алгоритмов, которые можно попробовать применить в этом задании.
* Использовать больше обучающих данных. В корпусе, которым мы пользуемся, только пара тысяч предложений, чего может не хватать для по-настоящему хорошей модели выравнивания. Разумеется, неразмеченных параллельных английско-чешских корпусов гораздо больше, поэтому можно воспользоваться ими. Хорошая точка для старта — данные с соревнования по машинному переводу  [воркшопа WMT](http://www.statmt.org/wmt20/translation-task.html).
* В языках часто существуют слова наподобие артиклей или предлогов, которым не соответствует ни одно слово в переводе. Все рассмотренные в рамках задания модели это не учитывают, возможно, добавление возможности перевода в «нулевой» токен улучшит качество модели (при тестировании такие выравнивания имеет смысл выбрасывать)

In [ ]:
# ┐_(ツ)_┌━☆ﾟ.*･｡ﾟ